In [1]:
# encoding: UTF-8
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import scipy.sparse as sp
from petsc4py import PETSc

In [2]:
class FieldSplitSolver(object):
    def __init__(self):
        self.mat = PETSc.Mat()
        
        self.ksp = PETSc.KSP()
        self.ksp.create(PETSc.COMM_WORLD)        
        
        self.options = PETSc.Options()
        self.options.setValue('fieldsplit_velocity_ksp_type', 'gmres')
        self.options.setValue('fieldsplit_velocity_pc_type', 'bjacobi')
        self.options.setValue('fieldsplit_pressure_pc_type', 'jacobi')
        self.options.setValue('fieldsplit_pressure_inner_ksp_type', 'preonly')
        self.options.setValue('fieldsplit_pressure_inner_pc_type', 'jacobi')
        self.options.setValue('fieldsplit_pressure_upper_ksp_type', 'preonly')
        self.options.setValue('fieldsplit_pressure_upper_pc_type', 'jacobi')
       
        #     self.options.setValue('ksp_type', 'fgmres')  
        #     self.options.setValue('pc_type', 'fieldsplit')  
        #     self.options.setValue('pc_fieldsplit_type', 'schur')  
        #     self.options.setValue('pc_fieldsplit_schur_fact_type', 'lower')      

        #self.options.setValue('pc_fieldsplit_detect_saddle_point', '')
    
        #self.options.setValue('ksp_monitor_solution', '')
    
        # http://www.mcs.anl.gov/petsc/petsc-current/docs/manualpages/KSP/KSPFischerGuessCreate.html#KSPFischerGuessCreate
        # -ksp_fischer_guess <model,size>
        #     self.options.setValue('ksp_fischer_guess', '1, 10')




    def SetupMatrix(self, A00, A01, A10, A11):        
        mats = [[A00, A01], [A10, A11]]
        
        NV = A00.size[1] 
        NP = A01.size[1]
        
        self.size = NP + NV
    
        iscol_0 = np.arange(0, NV, dtype=int)
        iscol_1 = np.arange(NV, NV + NP, dtype=int)  
        
        iscol_velocity = PETSc.IS()
        iscol_velocity.createGeneral(iscol_0, comm=PETSc.COMM_WORLD)
        
        iscol_pressure = PETSc.IS()
        iscol_pressure.createGeneral(iscol_1, comm=PETSc.COMM_WORLD)
        
        self.iscols = [iscol_velocity, iscol_pressure]    
        
        isrow_0 = np.arange(0, NV, dtype=int)
        isrow_1 = np.arange(NV, NV + NP, dtype=int)  
         
        isrow_velocity = PETSc.IS()
        isrow_velocity.createGeneral(isrow_0, comm=PETSc.COMM_WORLD)
         
        isrow_pressure = PETSc.IS()
        isrow_pressure.createGeneral(isrow_1, comm=PETSc.COMM_WORLD)
         
        self.isrows = [isrow_velocity, isrow_pressure]
        
        self.mat.createNest(mats=mats, isrows=self.isrows, iscols=self.iscols, comm=PETSc.COMM_WORLD)
        #self.mat.createNest(mats=mats, comm=PETSc.COMM_WORLD)
        
        self.sol, self.rhs = self.mat.createVecs()
        
        self.ksp.setType(PETSc.KSP.Type.FGMRES) 
        self.ksp.setOperators(self.mat, self.mat)
        
        self.pc = self.ksp.getPC()
        self.pc.setType(PETSc.PC.Type.FIELDSPLIT)            
        self.pc.setFieldSplitType(PETSc.PC.CompositeType.SCHUR) # schur == 4
        self.pc.setFieldSplitSchurFactType(PETSc.PC.SchurFactType.LOWER) # 'lower' == 1
#     self.pc.setFieldSplitSchurPreType(PETSc.PC.SchurPreType.SELFP)

        
        fields = [('velocity', iscol_velocity), ('pressure', iscol_pressure)]
        self.pc.setFieldSplitIS(*fields)  
        
        self.ksp.setUp()
#         subksps = self.pc.getFieldSplitSubKSP()
#         subksps[0].setType('gmres')
#         subksps[0].getPC().setType('bjacobi')
#         subksps[1].setType('preonly')
#         subksps[1].getPC().setType('jacobi')  
        self.mat.view()
        # Do I need this?
        #self.mat.assemblyBegin()
        #self.mat.assemblyEnd()
        
    
    def setup_vectors(self, rhs_vec_scipy, sol_vec_initial_values):         
         
        indices = np.arange(0, rhs_vec_scipy.shape[0], dtype=int)
        
        values  = rhs_vec_scipy
        self.rhs.setValues(indices, values)
        
        values  = sol_vec_initial_values
        self.sol.setValues(indices, values)


    def solve(self):
        self.ksp.setFromOptions()
        return self.ksp.solve(self.rhs, self.sol)

In [5]:
def StokesSetupVectors(nx, ny, hx, hy):
    #/* solution vector x */
    x = PETSc.Vec()
    x.create(PETSc.COMM_WORLD)
    x.setSizes(3*nx*ny, bsize=None)
    
#     x.setType(x.Type.MPI)
    x.setType(x.Type.SEQ)
   
    #/* exact solution y */
    y = x.duplicate()   
    y_array = y.getArray() 
    y_array = StokesExactSolution(nx, ny, hx, hy, y_array)
    
    #/* rhs vector b */
    b = x.duplicate()   
    b_array = b.getArray() 
    b_array = StokesRhs(nx, ny, hx, hy, b_array)
    
    return x, b



def StokesExactVelocityX(y):
    '''
    exact solution for the velocity (x-component, y-component is zero)
    [u1...uN v1...vN p1...pN]
    '''
    return 4.0*y*(1.0-y)


def StokesExactPressure(x):
    '''
    exact solution for the pressure
    '''
    return 8.0*(2.0-x)


def StokesGetPosition(nx, ny, idx):
    ''' 
    cell number n=j*nx+i has position (i,j) in grid 
    '''
    n  = idx % (nx*ny)
    i = n % nx
    j = (n - i)/nx
    return i, j

 
def StokesExactSolution(nx, ny, hx, hy, y_array):    
    
    
    # velocity part  
    u_idx = np.arange(nx*ny, dtype=int)
    i, j = StokesGetPosition(nx, ny, u_idx) 
    y_array[u_idx] = StokesExactVelocityX(j*hy + hy/2)

    # pressure part
    p_idx = np.arange(2*nx*ny, 3*nx*ny, dtype=int)
    i, j = StokesGetPosition(nx, ny, p_idx) 
    y_array[p_idx] = StokesExactPressure(i*hx + hx/2)
    
    return y_array
 
def StokesRhs(nx, ny, hx, hy, b_array):
    N = nx*ny
    # velocity part 
    u_idx = np.arange(N, dtype=int)
    i, j = StokesGetPosition(nx, ny, u_idx) 
    b_array[u_idx] = StokesRhsMomX(hx, hy, i, j)
    
    v_idx = np.arange(N, 2*N, dtype=int)    
    i, j = StokesGetPosition(nx, ny, v_idx) 
    b_array[v_idx] = StokesRhsMomY(hx, hy, i, j)
    
    # pressure part 
    p_idx = np.arange(2*N, 3*N, dtype=int)
    i, j = StokesGetPosition(nx, ny, p_idx) 
    b_array[p_idx] = StokesRhsMass(hx, hy, i, j)
    
    return b_array

def StokesRhsMomX(hx, hy, i, j):
    y   = j*hy + hy/2
    awb = hy / (hx/2)
    
    # west boundary
    return np.where(i == 0, awb*StokesExactVelocityX(y), 0.0)


def StokesRhsMomY(hx, hy, i, j):
    return 0.0
 
def StokesRhsMass(hx, hy, i, j):
    y   = j*hy + hy/2
    aeb = hy
    
    # west boundary
    return np.where(i == 0, aeb*StokesExactVelocityX(y), 0.0)


def view_matrix(mat):
    print(mat.todense())
    import matplotlib.pyplot as plt
    plt.spy(mat)
    plt.show()
    
def StokesSetupMatBlock00(nx, ny, hx, hy):   
    
    # A[0] is 2N-by-2N 
    subA = PETSc.Mat()
    subA.create(PETSc.COMM_WORLD)
    subA.setSizes([2*nx*ny,2*nx*ny])
    subA.setType('aij') # sparse
    subA.setPreallocationNNZ(5)
    
    # Create sparse matrix    
    N  = nx*ny
    
    aE = np.ones(N) * hy/hx
    aW = np.ones(N) * hy/hx
    
    aN = np.ones(N) * hx/hy
    aS = np.ones(N) * hx/hy
    
    # west boundary
    aWij = aW.reshape((ny, nx))
    aWij[:, 0] = 0.0
    aW = aWij.flatten()
    
    # east boundary
    aEij = aE.reshape((ny, nx))
    aEij[:, -1] = 0.0
    aE = aEij.flatten()
    
    # north boundary
    aN[np.arange(N-nx,N)] = 0.0
    
    # south boundary
    aS[np.arange(0,nx)] = 0.0
    
    aP = np.ones(N)*-(aE + aW + aS + aN)    
    # add boundary parcels
    
    aPij = aP.reshape((ny, nx))
    # west boundary
    aPij[:, 0] += -hy/(hx/2)   
    
    # east boundary
    aPij[:, -1] += -0*hy/(hx/2)   
 
    aP = aPij.flatten()
     
    # south boundary
    aP[np.arange(0,nx)] += -hx/(hy/2)     
    # north boundary
    aP[np.arange(N-nx,N)] += -hx/(hy/2)
    
    diagonals = [aP,aW[1:],aE,aN,aS[nx:]]
    laplacian = sp.diags(diagonals, [0, -1, 1, nx, -nx], format="csr")
    
    scipy_mat = sp.bmat([[laplacian, None], [None, laplacian]]).tocsr()
    
    
    I = scipy_mat.indptr # Index pointer
    J = scipy_mat.indices
    V = scipy_mat.data * (-1.0) # dynamic viscosity coef mu=-1 ?
    subA.setValuesCSR(I, J, V) 

    
    subA.assemblyBegin()
    subA.assemblyEnd()

    # view_matrix(scipy_mat)
    
    return subA


def StokesSetupMatBlock01(nx, ny, hx, hy):       
    # A[1] is 2N-by-N
    N  = nx*ny
    
    subA = PETSc.Mat()
    subA.create(PETSc.COMM_WORLD)
    subA.setSizes([2*N, N])
    subA.setType('aij') # sparse
    subA.setPreallocationNNZ(5)
    
    # Create sparse matrix
    aE = np.ones(N) *  hy/2
    aW = np.ones(N) * -hy/2
    
    # west boundary
    aWij = aW.reshape((ny, nx))
    aWij[:, 0] = 0.0
    aW = aWij.flatten()
    
    # east boundary
    aEij = aE.reshape((ny, nx))
    aEij[:, -1] = 0.0
    aE = aEij.flatten()    
   
    
    aP = np.ones(N)*-(aE + aW)    
    # add boundary parcels    
    aPij = aP.reshape((ny, nx))
    # west boundary
    aPij[:, 0] += 0.0    
    
    # east boundary
    aPij[:, -1] += -hy   
    
    
    diagonals = [aP,aW[1:],aE]
    laplacianX = sp.diags(diagonals, [0, -1, 1], format="csr")
    
    aN = np.ones(N) *  hx/2
    aS = np.ones(N) * -hx/2
    aS[np.arange(0,nx)] = 0.0
    aN[np.arange(N-nx,N)] = 0.0
    
    aP = np.ones(N)*-(aN + aS)  
    
    diagonals = [aP,aS[nx:],aN]
    laplacianY = sp.diags(diagonals, [0, -nx, nx], format="csr")
    
    scipy_mat = sp.bmat([[laplacianX], [laplacianY]]).tocsr()

    I = scipy_mat.indptr # Index pointer
    J = scipy_mat.indices
    V = scipy_mat.data
    subA.setValuesCSR(I, J, V) 

    
    subA.assemblyBegin()
    subA.assemblyEnd()
    
    # view_matrix(scipy_mat)
    
    return subA, scipy_mat


def StokesSetupMatBlock10(nx, ny, scipy_mat):
    # A[2] is minus transpose of A[1]    
    N  = nx*ny
    
    subA = PETSc.Mat()
    subA.create(PETSc.COMM_WORLD)
    subA.setSizes([N, 2*N])
    subA.setType('aij') # sparse
    subA.setPreallocationNNZ(6)    
    
    scipy_mat_T = scipy_mat.transpose(copy=True).tocsr()
    I = scipy_mat_T.indptr # Index pointer
    J = scipy_mat_T.indices
    V = scipy_mat_T.data * (-1.0)
    subA.setValuesCSR(I, J, V) 
    
    subA.assemblyBegin()
    subA.assemblyEnd()

    # view_matrix(scipy_mat_T)
    
    return subA

 
def StokesSetupMatBlock11(nx, ny, hx, hy):
    # A[3] is N-by-N null matrix       
    N  = nx*ny
    
    subA = PETSc.Mat()
    subA.create(PETSc.COMM_WORLD)
    subA.setSizes([N, N])
    subA.setType('aij') # sparse
    subA.setPreallocationNNZ(1)
    
    scipy_mat = sp.csr_matrix((N, N))
    I = scipy_mat.indptr # Index pointer
    J = scipy_mat.indices
    V = scipy_mat.data 
    subA.setValuesCSR(I, J, V) 
    
    subA.assemblyBegin()
    subA.assemblyEnd()
    
    return subA


def StokesSetupMatrix(nx, ny, hx, hy):
    subA00 = StokesSetupMatBlock00(nx, ny, hx, hy)
    subA01, scipy_mat = StokesSetupMatBlock01(nx, ny, hx, hy)
    subA10 = StokesSetupMatBlock10(nx, ny, scipy_mat)
    subA11 = StokesSetupMatBlock11(nx, ny, hx, hy)
    
    mats = [[subA00, subA01], [subA10, subA11]]
    
    mat = PETSc.Mat()
    N = nx * ny

    iscols0 = np.arange(0, 2*N, dtype=int)
    iscols1 = np.arange(2*N, 3*N, dtype=int)  
    
    petscIS0 = PETSc.IS()
    petscIS0.createGeneral(iscols0, comm=PETSc.COMM_WORLD)
    
    petscIS1 = PETSc.IS()
    petscIS1.createGeneral(iscols1, comm=PETSc.COMM_WORLD)
    
    iscols = [petscIS0, petscIS1]    
    
    isrows0 = np.arange(0, 2*N, dtype=int)
    isrows1 = np.arange(2*N, 3*N, dtype=int)  
     
    petscISrow0 = PETSc.IS()
    petscISrow0.createGeneral(isrows0, comm=PETSc.COMM_WORLD)
     
    petscISrow1 = PETSc.IS()
    petscISrow1.createGeneral(isrows1, comm=PETSc.COMM_WORLD)
     
    isrows = [petscISrow0, petscISrow1]     
    
#     mat.createNest(mats=mats, isrows=isrows, iscols=iscols, comm=PETSc.COMM_WORLD)
    mat.createNest(mats=mats, comm=PETSc.COMM_WORLD)
    mat.assemblyBegin()
    mat.assemblyEnd()

    #StokesSetupApproxSchur(s);CHKERRQ(ierr);
    return mat, isrows



def test_field_split(): 
    nx = 4
    ny = 4

    hx = 2.0/nx
    hy = 1.0/ny    

    options = PETSc.Options()
    options.setValue('fieldsplit_velocity_ksp_type', 'gmres')
    options.setValue('fieldsplit_velocity_pc_type', 'bjacobi')
    options.setValue('fieldsplit_pressure_pc_type', 'jacobi')
    options.setValue('fieldsplit_pressure_inner_ksp_type', 'preonly')
    options.setValue('fieldsplit_pressure_inner_pc_type', 'jacobi')
    options.setValue('fieldsplit_pressure_upper_ksp_type', 'preonly')
    options.setValue('fieldsplit_pressure_upper_pc_type', 'jacobi')
       
#     options.setValue('ksp_type', 'fgmres')  
#     options.setValue('pc_type', 'fieldsplit')  
#     options.setValue('pc_fieldsplit_type', 'schur')  
#     options.setValue('pc_fieldsplit_schur_fact_type', 'lower')      

    options.setValue('pc_fieldsplit_detect_saddle_point', '')
    
    #options.setValue('ksp_monitor_solution', '')
    
    # http://www.mcs.anl.gov/petsc/petsc-current/docs/manualpages/KSP/KSPFischerGuessCreate.html#KSPFischerGuessCreate
    # -ksp_fischer_guess <model,size>
#     options.setValue('ksp_fischer_guess', '1, 10')
 
    A, isrows = StokesSetupMatrix(nx, ny, hx, hy)

    x, b = StokesSetupVectors(nx, ny, hx, hy)
    
    ksp = PETSc.KSP()
    ksp.create(PETSc.COMM_WORLD)
    ksp.setType(PETSc.KSP.Type.FGMRES)
    ksp.setOperators(A, A)
        
    # StokesSetupPC
    pc = ksp.getPC()
    pc.setType(PETSc.PC.Type.FIELDSPLIT)  
      
    pc.setFieldSplitType(PETSc.PC.CompositeType.SCHUR) # schur == 4
    pc.setFieldSplitSchurFactType(PETSc.PC.SchurFactType.LOWER) # 'lower' == 1
#     pc.setFieldSplitSchurPreType(PETSc.PC.SchurPreType.SELFP)
    
    fields = [('velocity', isrows[0]), ('pressure', isrows[1])]
    pc.setFieldSplitIS(*fields)    

    ksp.setUp()
#     subksps = pc.getFieldSplitSubKSP()
#     subksps[0].setType('gmres')
#     subksps[0].getPC().setType('bjacobi')
#     subksps[1].setType('preonly')
#     subksps[1].getPC().setType('jacobi')

    ksp.setFromOptions()
    ksp.solve(b, x)
    
    x_sol = np.array(x)
    N = nx * ny
    U = x_sol[0:N].reshape((ny,nx))
    V = x_sol[N:2*N].reshape((ny,nx))
    P = x_sol[2*N:3*N].reshape((ny,nx))
    
    x = np.linspace(0, 2.0, num=nx)
    y = np.linspace(0, 1.0, num=ny)
    X, Y = np.meshgrid(x,y)
    
    
    import matplotlib.pyplot as plt
    plt.figure()
    CS = plt.contourf(X, Y, P, 200)
    
    plt.quiver(X, Y, U, V,        # data
               headlength=7)        # length of the arrows
    
    
    plt.show()

nx = 4
ny = 6
N = nx*ny

hx = 2.0/nx
hy = 1.0/ny
A00 = StokesSetupMatBlock00(nx, ny, hx, hy)
A01, scipy_mat = StokesSetupMatBlock01(nx, ny, hx, hy)
A10 = StokesSetupMatBlock10(nx, ny, scipy_mat)
A11 = StokesSetupMatBlock11(nx, ny, hx, hy)    

x0  = np.zeros(3*N)
x0 = StokesExactSolution(nx, ny, hx, hy, x0)  
rhs = np.zeros(3*N)
rhs = StokesRhs(nx, ny, hx, hy, rhs)


solver = FieldSplitSolver()
solver.SetupMatrix(A00, A01, A10, A11)
solver.setup_vectors(rhs_vec_scipy=rhs, sol_vec_initial_values=x0)
solver.solve()

solver.sol[...]

array([  3.18366479e-01,   3.19541379e-01,   3.20155812e-01,
         3.20165964e-01,   7.47128481e-01,   7.47162114e-01,
         7.47070190e-01,   7.47076649e-01,   9.62282934e-01,
         9.61074506e-01,   9.60552004e-01,   9.60535277e-01,
         9.62282937e-01,   9.61074489e-01,   9.60551955e-01,
         9.60535294e-01,   7.47128480e-01,   7.47162069e-01,
         7.47070086e-01,   7.47076695e-01,   3.18366478e-01,
         3.19541343e-01,   3.20155729e-01,   3.20166003e-01,
        -3.61175087e-03,  -2.05379583e-04,  -7.89968901e-05,
        -2.04597279e-07,  -5.32027454e-03,  -3.90839173e-04,
        -1.29618464e-04,  -2.73919639e-06,  -1.70841835e-03,
        -1.85321811e-04,  -5.06133175e-05,  -2.72542949e-06,
         1.70881019e-03,   1.85811455e-04,   5.06115762e-05,
         2.02510291e-06,   5.32057516e-03,   3.91191384e-04,
         1.29595918e-04,   2.21971329e-06,   3.61187030e-03,
         2.05517735e-04,   7.89926070e-05,   3.78164826e-09,
         1.52837647e+01,